<a href="https://colab.research.google.com/github/MrKhan141/Automated-Identification-Of-fish-species/blob/main/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
import pandas as pd
import numpy as np
from keras.utils import np_utils
from pathlib import Path
import tensorflow
from tensorflow.keras.preprocessing import image

In [ ]:
p = Path('/content/drive/MyDrive/Fish_Dataset/')
dirs = p.glob('*')

In [ ]:
image_data = []
image_path = []
labels = []
label_dict = {"Black Sea Sprat":0,"Gilt-Head Bream":1 ,"Hourse Mackerel":2 ,"Red Mullet":3 ,"Red Sea Bream": 4 , "Sea Bass": 5 ,"Shrimp": 6,"Striped Red Mullet": 7,"Trout": 8}
label2fish = {0:"Black Sea Sprat",1:"Gilt-Head Bream",2:"Hourse Mackerel",3:"Red Mullet",4:"Red Sea Bream",5:"Sea Bass" ,6:"Shrimp",7:"Striped Red Mullet",8:"Trout"}
for folder_dir in dirs :
    label=str(folder_dir).split('/')[-1]
    folder=label
    cnt = 0 
    print(folder)
    for img_path  in folder_dir.glob("*.png"):
        img = image.load_img(img_path,target_size =(128,128))
        img_array = image.img_to_array(img)
        image_data.append(img_array)
        labels.append(label_dict[label])
        cnt += 1
    print(cnt)

Striped Red Mullet
1000
Shrimp
1000
Trout
1000
Sea Bass
880
Black Sea Sprat
1000
Gilt-Head Bream
1000
Hourse Mackerel
1000
Red Mullet
1000
Red Sea Bream
1000


In [ ]:
import random
combined = list(zip(image_data,labels))
random.shuffle(combined)
image_data[:],labels[:]=zip(*combined)

In [ ]:
labels[:5]

[0, 8, 5, 2, 3]

In [ ]:
X = np.array(image_data)
# Normalization 
X_Train = X/255.0
Y_Train = np.array(labels)
Y_Train = np_utils.to_categorical(Y_Train,9)
print(X_Train.shape,Y_Train.shape)

(8880, 128, 128, 3) (8880, 9)


In [ ]:
# Create ResNet50 model
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.layers import *
import matplotlib.pyplot as plt

In [ ]:
model=ResNet50(include_top=False,weights='imagenet',input_shape=(128,128,3))

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']      

In [ ]:
from keras.models import Model

In [ ]:
av1=GlobalAveragePooling2D()(model.output)
fc1 = Dense(256,activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(9,activation='softmax')(d1)
new_model = Model(inputs = model.input,outputs= fc2)
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']       

In [ ]:
adam=Adam(learning_rate=0.0002)
new_model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [ ]:
for i in range(169):
  new_model.layers[i].trainable=False
new_model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [ ]:
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']       

In [ ]:
hist = new_model.fit(X_Train,Y_Train,shuffle=True,batch_size=256,epochs=32,validation_split=0.20)

Epoch 1/32
28/28 [==============================] - 529s 19s/step - loss: 1.8772 - accuracy: 0.3205 - val_loss: 2.2454 - val_accuracy: 0.1092
Epoch 2/32
28/28 [==============================] - 511s 18s/step - loss: 1.4353 - accuracy: 0.5218 - val_loss: 2.2518 - val_accuracy: 0.1132
Epoch 3/32
28/28 [==============================] - 512s 18s/step - loss: 1.1909 - accuracy: 0.6080 - val_loss: 2.2588 - val_accuracy: 0.1588
Epoch 4/32
28/28 [==============================] - 514s 18s/step - loss: 1.0276 - accuracy: 0.6582 - val_loss: 2.2611 - val_accuracy: 0.1149
Epoch 5/32
28/28 [==============================] - 515s 18s/step - loss: 0.8936 - accuracy: 0.7047 - val_loss: 2.2609 - val_accuracy: 0.1194
Epoch 6/32
28/28 [==============================] - 514s 19s/step - loss: 0.7693 - accuracy: 0.7459 - val_loss: 2.2532 - val_accuracy: 0.1306
Epoch 7/32
28/28 [==============================] - 513s 18s/step - loss: 0.6902 - accuracy: 0.7779 - val_loss: 2.2263 - val_accuracy: 0.1408
Epoch 

In [ ]:
from tensorflow.keras.models import load_model
new_model.save('/content/drive/MyDrive/save_model/weight_gain.json')

In [ ]:
model_json=new_model.to_json()
with open("new_model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
test = X[15]
test = test.reshape(1,128,128,3)
print(test.shape)

(1, 128, 128, 3)


In [ ]:
re=new_model.predict(test)
re = np.argmax(re)
sre = label2fish[re]
print(sre)

1/1 [==============================] - 0s 116ms/step
Red Sea Bream
